### Часть 2: Попробуйте частотный анализ для биграм

Вряд ли в части 1 получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п. 1, но при помощи биграмм. 

В качестве естественной метрики качества можно взять долю правильно расшифрованных букв или, если хочется совсем математически изощриться, расстояние между двумя перестановками, правильной и полученной из модели; но, честно говоря, в этом задании следить за численными метриками не так уж обязательно, будет и глазами всё видно.

In [1]:
import numpy as np
from nltk import ngrams
from collections import Counter

from utils import get_corpus, clean_text, encode_mapping, apply_mapping, ngram_freq_dict, accuracy

np.random.seed(5012)

In [2]:
corpus = get_corpus()
text = clean_text('Сама того не желая, Анна всем приносит несчастье. И Каренину, который говорит: «я убит, я разбит, я не человек больше», и Вронскому, который признается: «как человек – я развалина», и самой себе: «была ли когда-нибудь женщина так несчастна, как я», – восклицает она. И то, что она бросается поперек дороги под колеса неумолимо надвигающегося на нее вагона, тоже получило в романе символический смысл. Конечно, смысл трагедии не в самой железной дороге… Но «звезда Полынь» заключала в себе такой заряд современной поэзии, что Толстой, как поэт и романист, не мог пренебречь ее художественными возможностями.')

In [3]:
freqs = ngram_freq_dict(corpus, n_gram=1)
encoded_text = apply_mapping(text, encode_mapping(freqs))

In [4]:
corpus_freqs_bigram = ngram_freq_dict(corpus, n_gram=2)
text_freqs_bigram = ngram_freq_dict(encoded_text, n_gram=2)

In [10]:
def create_decode_mapping_ngram(corpus_freqs, text_freqs, ngram):
    decode_mapping = {}
    used_corpus_ngrams = set()
    for text_ngram, text_ngram_freq in text_freqs.items():
        # check if already decoded
        decoded_ngram = [c if c in decode_mapping else None for c in text_ngram]
        if None not in decoded_ngram:
            continue
        
        # find best match (closest freq) in group of simular ngrams
        min_diff = 1
        best_freq = None
        best_ngram = None
        for corpus_ngram, corpus_ngram_freq in corpus_freqs.items():
            # skip if ngram has been used before
            if corpus_ngram in used_corpus_ngrams:
                continue
            # skip if bigram is simular
            skip_cond = False
            for i in range(ngram):
                if (decoded_ngram[i] is not None and decoded_ngram[i] != corpus_ngram[i]):
                    skip_cond = True
                if (decoded_ngram[i] is None and corpus_ngram[i] in decode_mapping.values()):
                    skip_cond = True
            if skip_cond:
                continue
            # choose closest from the rest
            diff = abs(corpus_ngram_freq - text_ngram_freq)
            if diff < min_diff:
                best_ngram = corpus_ngram
                best_freq = corpus_ngram_freq
                min_diff = diff

        # save seen ngram
        used_corpus_ngrams.add(best_ngram)
        
        # save best result
        for j in range(ngram):
            if best_ngram is not None and text_ngram[j] not in decode_mapping:
                decode_mapping[text_ngram[j]] = best_ngram[j]

    return decode_mapping

In [11]:
decode_mapping = create_decode_mapping_ngram(corpus_freqs_bigram, text_freqs_bigram, 2)
decoded_text = apply_mapping(encoded_text, decode_mapping)
decoded_text

'чояо нака се зепоы оссо лчея тьисачин сечшочнэе и доьесисй данаьюу калаьин ы ймин ы ьоцмин ы се шепалед мапэбе и льасчдаяй данаьюу тьицсоенчы дод шепалед ы ьоцлописо и чояау чеме мюпо пи даквосимйвэ зесжисо нод сечшочнсо дод ы лачдпироен асо и на шна асо мьачоенчы татеьед ваьаки тав дапечо сейяапияа совликощжекачы со сее локасо назе тапйшипа л ьаяосе чиялапишечдиу чяючп дасешса чяючп ньокевии се л чояау зепецсау ваьаке са цлецво тапюсэ цодпщшопо л чеме нодау цоьыв чальеяессау тагции шна напчнау дод тагн и ьаяосичн се яак тьесемьешэ ее фйвазечнлессюяи лацяазсачныяи'

In [8]:
text

'сама того не желая анна всем приносит несчастье и каренину который говорит я убит я разбит я не человек больше и вронскому который признается как человек я развалина и самой себе была ли когданибудь женщина так несчастна как я восклицает она и то что она бросается поперек дороги под колеса неумолимо надвигающегося на нее вагона тоже получило в романе символический смысл конечно смысл трагедии не в самой железной дороге но звезда полынь заключала в себе такой заряд современной поэзии что толстой как поэт и романист не мог пренебречь ее художественными возможностями'

In [9]:
accuracy(text, decoded_text)

0.2964912280701754